In [11]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import statistics
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

# Connect to database
#conn = sqlite3.connect("/home/runner/work/aiap20-jaye-lin-jiaqi-945I/aiap20-jaye-lin-jiaqi-945I/data/bmarket.db")
conn = sqlite3.connect("/Users/jayelin/Downloads/docker_image/data/bmarket.db")

# Fetch all rows from the 'bank_marketing' table
cursor = conn.cursor()

cursor.execute("SELECT * FROM bank_marketing") # cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
rows = cursor.fetchall()

# Create DataFrame from the results
columns = [description[0] for description in cursor.description]  # Get column names
df = pd.DataFrame(rows, columns=columns)

# Data cleaning
def yes_no_binary(yes_no_str):
    
    if yes_no_str == 'no':
        return 0
    elif yes_no_str == 'yes':
        return 1
    else:
        return -1
    
def mobile_binary(mobile_str):
    
    if mobile_str == 'Telephone':
        return 'telephone'
    elif mobile_str == 'Cell':
        return 'cellular'
    else:
        return mobile_str

df['Housing Loan'] = df['Housing Loan'].apply(yes_no_binary)
df['Personal Loan'] = df['Personal Loan'].apply(yes_no_binary)
df['Subscription Status'] = df['Subscription Status'].apply(yes_no_binary)
df['Credit Default'] = df['Credit Default'].apply(yes_no_binary)
df['Contact Method'] = df['Contact Method'].apply(mobile_binary)
df.head()

# Feature engineering
def contact_binary(str):
    if str == 'telephone':
        return 1
    elif str == 'cellular':
        return 2

def occupation_binary(str):
    if str == 'technician':
        return 1
    elif str == 'blue-collar':
        return 2
    elif str == 'admin.':
        return 3
    elif str == 'housemaid':
        return 4
    elif str == 'retired':
        return 5
    elif str == 'services':
        return 6
    elif str == 'entrepreneur':
        return 7
    elif str == 'unemployed':
        return 8
    elif str == 'management':
        return 9
    elif str == 'self-employed':
        return 10
    elif str == 'student':
        return 11
    elif str == 'unknown':
        return -1
    
def marital_binary(str):
    if str == 'married':
        return 1
    elif str == 'divorced':
        return 2
    elif str == 'single':
        return 3
    elif str == 'unknown':
        return -1

def education_binary(str):
    if str == 'illiterate':
        return 1
    elif str == 'basic.4y':
        return 2
    elif str == 'basic.6y':
        return 3
    elif str == 'basic.9y':
        return 4
    elif str == 'high.school':
        return 5
    elif str == 'professional.course':
        return 6
    elif str == 'university.degree':
        return 7
    elif str == 'unknown':
        return -1
    
def previous_contact_days_binary(str):
    if str == 999:
        return -1
    else:
        return str

df['Age'] = df['Age'].str.extract(r'(\d+)').astype(int)
df['Contact Method'] = df['Contact Method'].apply(contact_binary)
df['Occupation'] = df['Occupation'].apply(occupation_binary)
df['Marital Status'] = df['Marital Status'].apply(marital_binary)
df['Education Level'] = df['Education Level'].apply(education_binary)
df['Previous Contact Days'] = df['Previous Contact Days'].apply(previous_contact_days_binary)
df.head(10)

,Client ID,Age,Occupation,Marital Status,Education Level,Credit Default,Housing Loan,Personal Loan,Contact Method,Campaign Calls,Previous Contact Days,Subscription Status
0,32885,57,1,1,5,0,0,1,2,1,-1,0
1,3170,55,-1,1,-1,-1,1,0,1,2,-1,0
2,32207,33,2,1,4,0,0,0,2,1,-1,0
3,9404,36,3,1,5,0,0,0,1,4,-1,0
4,14021,27,4,1,5,0,-1,0,2,2,-1,0
5,17202,58,5,1,6,0,-1,1,2,1,-1,0
6,880,48,6,1,5,-1,1,0,1,1,-1,0
7,23758,150,3,2,7,-1,1,-1,2,7,-1,0
8,10822,24,7,1,7,0,-1,-1,1,4,-1,0
9,14356,36,1,2,6,0,1,1,2,4,-1,0


In [ ]:
# Build machine learning models and evaluate with AUC metric
X = df.drop(columns='Subscription Status')
y = df['Subscription Status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Instantiate the model object
rf_model = RandomForestClassifier(random_state=5, n_estimators=300, max_depth=20, min_samples_leaf=4)

#xgb_model.fit(X_train,y_train)
rf_model.fit(X_train, y_train)

# Predict the target on the test dataset
rf_preds = rf_model.predict_proba(X_test)[:, 1]

# Calculate the AUC-ROC score on the test set
auc = roc_auc_score(y_test, rf_preds)
print('\nAUC-ROC score on test set:', auc)

print(rf_preds)



AUC-ROC score on test set: 0.7804181372675835
[0.05187892 0.35768139 0.45944421 ... 0.2328678  0.04732865 0.13001042]


In [ ]:

# Build machine learning models and evaluate with AUC metric

X = df.drop(columns='Subscription Status')
y = df['Subscription Status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Instantiate the model object
knn_model = KNeighborsClassifier(n_neighbors=20, weights='distance', metric='manhattan')

knn_model.fit(X_train, y_train)

# Predict the target on the test dataset
knn_preds = knn_model.predict_proba(X_test)[:, 1]

# Calculate the AUC-ROC score on the test set
auc = roc_auc_score(y_test, knn_preds)
print('\nAUC-ROC score on test set:', auc)

print(knn_preds)


AUC-ROC score on test set: 0.7775593260428485
[0.         0.13108773 0.57042423 ... 0.17940847 0.09928854 0.08286494]


In [ ]:
# Build machine learning models and evaluate with AUC metric

X = df.drop(columns='Subscription Status')
y = df['Subscription Status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Instantiate the model object
lr_model = LogisticRegression(max_iter=300)

lr_model.fit(X_train, y_train)

# Predict the target on the test dataset
lr_preds = knn_model.predict_proba(X_test)[:, 1]

# Calculate the AUC-ROC score on the test set
auc = roc_auc_score(y_test, lr_preds)
print('\nAUC-ROC score on test set:', auc)

print(lr_preds)



AUC-ROC score on test set: 0.7775593260428485
[0.         0.13108773 0.57042423 ... 0.17940847 0.09928854 0.08286494]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
